In [1]:
import yfinance as yf
from datetime import datetime

In [2]:
def get_acao(mercado,acao,inicio,final):
    if mercado == 'Brasil':
        acao = acao +'.SA'
    else:
        acao = acao
    dat = yf.Ticker(acao)
    data = dat.history(start=inicio,end=final)
    data = data.reset_index()
    data = data[['Date','Close']]
    return data

In [5]:
inicio = datetime(2024,12,15)
final = datetime(2025,3,3)
data = get_acao('Brasil','PETR4',inicio,final)

In [30]:
dat = yf.Ticker('VALE3.SA')
dict_info = dat.info

In [46]:
# Lista com as chaves que você quer manter
chaves_desejadas = ['address1','city', 'country','state','phone','website','industry',
 'industryKey','sectorKey','longBusinessSummary','fullTimeEmployees','overallRisk']

# Novo dicionário com apenas essas chaves
novo_dict = {chave: dict_info[chave] for chave in chaves_desejadas}

In [47]:
novo_dict

{'address1': 'Praia de Botafogo 186',
 'city': 'Rio De Janeiro',
 'country': 'Brazil',
 'state': 'RJ',
 'phone': '55 21 3485 5000',
 'website': 'https://www.vale.com',
 'industry': 'Other Industrial Metals & Mining',
 'industryKey': 'other-industrial-metals-mining',
 'sectorKey': 'basic-materials',
 'longBusinessSummary': 'Vale S.A., together with its subsidiaries, produces and sells iron ore, iron ore pellets, nickel, and copper in Brazil and internationally. The company operates through Iron Solutions and Energy Transition Materials segments. The Iron Solutions segment produces and extracts iron ore and pellets, and other ferrous products; and provides related logistic services. The Energy Transition Materials segment produces and extracts nickel; and its by-products, such as gold, silver, cobalt, precious metals, and others, as well as copper used in the construction sector to produce pipes and electrical wires. The company was formerly known as Companhia Vale do Rio Doce and change

In [34]:
lista_chaves = list(dict_info.keys())

In [ ]:
'address1','city', 'country','state',
 'phone',
 'website',
 'industry',
 'industryKey',

 'sectorKey',
 'longBusinessSummary',
 'fullTimeEmployees',
 'overallRisk',

In [1]:
lista_acao = ['PETR4','VALE3']

In [5]:
mercado = 'Brasil'
data_inicio = datetime(2025,4,1)
data_final = datetime(2025,4,7)

In [8]:
import pandas as pd
df_acoes = pd.DataFrame()
for acao in lista_acao:
        df_precos = get_acao(mercado,acao,data_inicio,data_final)
        df_precos['ativo'] = acao
        df_precos['Date'] = pd.to_datetime(df_precos['Date']).dt.strftime('%d/%m/%Y')
        df_acoes = pd.concat([df_acoes,df_precos])

In [18]:
def get_bench(bench,inicio,final):
    dat = yf.Ticker(bench)
    data = dat.history(start=inicio,end=final)
    data = data.reset_index()
    data['ativo'] = bench
    data = data[['Date','ativo','Close']]
    return data


bench = get_bench('^BVSP',data_inicio,data_final)
bench_name = '^BVSP'


In [19]:
bench['Date'] = pd.to_datetime(bench['Date']).dt.strftime('%d/%m/%Y')

In [20]:
df_junto = pd.concat([df_acoes,bench])

In [22]:
df_pivot = df_junto.pivot(index='Date', columns='ativo', values='Close')
    

In [23]:
df_pivot

ativo,PETR4,VALE3,^BVSP
Date,,,
01/04/2025,37.299999,57.189999,131147.0
02/04/2025,37.200001,56.930000,131190.0
03/04/2025,36.000000,54.869999,131141.0
04/04/2025,34.549999,52.680000,127256.0


In [24]:
returns = df_pivot.pct_change().dropna()
    

In [25]:
returns

ativo,PETR4,VALE3,^BVSP
Date,,,
02/04/2025,-0.002681,-0.004546,0.000328
03/04/2025,-0.032258,-0.036185,-0.000374
04/04/2025,-0.040278,-0.039912,-0.029625


In [26]:
benchmark_name = '^BVSP'

ret_benchmark = returns[benchmark_name]



In [27]:
ret_benchmark

Date
02/04/2025    0.000328
03/04/2025   -0.000374
04/04/2025   -0.029625
Name: ^BVSP, dtype: float64

In [28]:
import numpy as np
betas = {}

for ativo in returns.columns:
    if ativo != benchmark_name:
        cov = np.cov(returns[ativo], ret_benchmark)[0][1]
        var = np.var(ret_benchmark)
        beta = cov / var
        betas[ativo] = beta
df_betas = pd.DataFrame.from_dict(betas, orient='index', columns=['Beta'])

In [31]:
df_betas.reset_index()

,index,Beta
0,PETR4,1.181892
1,VALE3,1.018557


In [3]:
def calcula_retorno(df,data_fim):
    data_busca = pd.Timestamp(data_fim, tz="America/Sao_Paulo")
    retorno = (df['Close'].iloc[-1] / df.loc[df['Date'] == data_busca]['Close'].iloc[0])-1
    return retorno

In [7]:
import pandas as pd
data_final = datetime(2025,2,10)
retorno = calcula_retorno(data,data_final)

In [16]:
lista_acao = ['PETR4','VALE3']
mercado = 'Brasil'
data_inicio = datetime(2025,4,1)
data_final = datetime(2025,4,12)


In [38]:
df_acoes = pd.DataFrame()
if len(lista_acao)>=1:
    for acao in lista_acao:
        df_precos = get_acao(mercado,acao,data_inicio,data_final)
        df_precos['ativo'] = acao
        df_precos['Date'] = pd.to_datetime(df_precos['Date']).dt.strftime('%d/%m/%Y')
        df_acoes = pd.concat([df_acoes,df_precos])

In [ ]:
def calcula_retorno(df,data_fim):
    n_acoes = df['ativo'].unique().tolist()
    df_retornos = pd.DataFrame()
    for ativo in n_acoes:
        data_busca = pd.Timestamp(data_fim, tz="America/Sao_Paulo")
        df_copy = df[df['ativo'] == ativo]
        retorno = (df_copy['Close'].iloc[-1] / df_copy.loc[df_copy['Date'] == data_busca]['Close'].iloc[0])-1
        df_nova = pd.DataFrame({'ativo':[ativo],'retorno':[retorno]})
        if len(df_retornos>1):
            df_retornos = pd.concat([df_retornos,df_nova])
        else:
            df_retornos = df_nova
    return df_retornos


In [19]:
n_acoes = df_acoes['ativo'].unique().tolist()
df_retornos = pd.DataFrame()

In [ ]:
data_fim = datetime(2025,4,4)
df_retornos = pd.DataFrame()
for ativo in n_acoes:
    data_busca = pd.to_datetime(data_fim)
    df_copy = df[df['ativo'] == ativo]
    retorno = (df_copy['Close'].iloc[-1] / df_copy.loc[df_copy['Date'] == data_busca]['Close'].iloc[0])-1
    df_nova = pd.DataFrame({'ativo':[ativo],'retorno':[retorno]})
    if len(df_retornos>1):
        df_retornos = pd.concat([df_retornos,df_nova])
    else:
        df_retornos = df_nova
return df_retornos

IndexError: single positional indexer is out-of-bounds

In [35]:
df_copy
(df_copy['Close'].iloc[-1] / df_copy.loc[df_copy['Date'] == data_busca]['Close'].iloc[0])-1

-0.07814758075868866

In [29]:
data_busca = pd.to_datetime(data_fim)

In [31]:
data_busca

Timestamp('2025-04-04 00:00:00')

In [33]:
df_copy['Date'] = pd.to_datetime(df_copy['Date'])

C:\Users\phcer\AppData\Local\Temp\ipykernel_26344\1967456879.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['Date'] = pd.to_datetime(df_copy['Date'])


In [36]:
df_retornos = pd.DataFrame()
df_nova = pd.DataFrame({'ativo':[ativo],'retorno':[retorno]})
if len(df_retornos>1):
    df_retornos = pd.concat([df_retornos,df_nova])
else:
    df_retornos = df_nova

In [39]:
n_acoes = df_acoes['ativo'].unique().tolist()
df_retornos = pd.DataFrame()


In [44]:
df_acoes['Date'] = pd.to_datetime(df_acoes['Date'])

In [46]:
for ativo in n_acoes:
    data_busca = pd.to_datetime(data_fim)
    df_copy = df_acoes[df_acoes['ativo'] == ativo]
    retorno = (df_copy['Close'].iloc[-1] / df_copy.loc[df_copy['Date'] == data_busca]['Close'].iloc[0])-1
    df_nova = pd.DataFrame({'ativo':[ativo],'retorno':[retorno]})
    

In [47]:
df_nova

,ativo,retorno
0,VALE3,0.018603


In [ ]:
if len(df_retornos)>1:
        df_retornos = pd.concat([df_retornos,df_nova])
    else:
        df_retornos = df_nova

In [55]:
from dateutil.relativedelta import relativedelta

data_periodo = data_final - relativedelta(years=1)

for acao in lista_acao:
    df_precos = get_acao(mercado,acao,data_periodo,data_final)
    df_precos['ativo'] = acao
    df_precos['Date'] = pd.to_datetime(df_precos['Date']).dt.strftime('%d/%m/%Y')
    df_acoes = pd.concat([df_acoes,df_precos])


In [54]:
data_periodo = data_final - relativedelta(years=1)

In [57]:
df_acoes['Date'] = pd.to_datetime(df_acoes['Date'])

In [58]:
df_acoes

,Date,Close,ativo
0,2025-01-04,37.299999,PETR4
1,2025-02-04,37.200001,PETR4
2,2025-03-04,36.000000,PETR4
3,2025-04-04,34.549999,PETR4
4,2025-07-04,33.180000,PETR4
...,...,...,...
246,2025-07-04,52.049999,VALE3
247,2025-08-04,49.200001,VALE3
248,2025-09-04,51.849998,VALE3
249,2025-10-04,52.779999,VALE3


In [ ]:
def calcula_retorno(df,data_fim):
    n_acoes = df['ativo'].unique().tolist()
    df_retornos = pd.DataFrame()
    for ativo in n_acoes:
        data_busca = pd.to_datetime(data_fim)
        df_copy = df[df['ativo'] == ativo]
        for dias_atras in range(4):
            data_busca = pd.to_datetime(data_fim - relativedelta(days=dias_atras))
            filtro = df_copy.loc[df_copy['Date'] == data_busca]

            if not filtro.empty:
                preco_inicial = filtro['Close'].iloc[0]
                preco_final = df_copy['Close'].iloc[-1]
                retorno = (preco_final / preco_inicial) - 1

        df_nova = pd.DataFrame({'ativo':[ativo],'retorno':[retorno]})
        if len(df_retornos)>= 1:
            df_retornos = pd.concat([df_retornos,df_nova])
        else:
            df_retornos = df_nova
    return df_retornos